<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_14_Traders_trade_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 13 and intro

Script Notes: 

In lesson 13 traders found their Von Nuemann trader neighbors, now in this lesson the agent and the neighbor agents need to determine if they should trade, so first they need a way to determine if trading will increase their welfare. In this lesson we will develop that function.  

*Review each class and model set up*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import math
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 2.5 MB 4.9 MB/s 
     |████████████████████████████████| 66 kB 3.9 MB/s 


In [3]:
def get_distance(pos_1, pos_2):
    """Calculate Euclidean distance between two positions.
    Args:
        pos_1, pos_2: Coordinate tuples for both points.
    
    used in trader.move()
    """
    x1, y1 = pos_1
    x2, y2 = pos_2
    dx = x1 - x2
    dy = y1 - y2
    return math.sqrt(dx**2 + dy**2)

In [4]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      
      

In [5]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn. (rule G1)
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
       

## Add Trade 

**Part 0**

------------

If you remember from last lesson we have list of agent objects who we can potentially trade with. We now want to iterate through that list and trade, as the trading is an involved process we are going to put trade in some layered 
functions. 

We will put our trade function above the main function block and it is important to point out that we passed in a parameter which is the agent object in the list and we have the agent or self who is initiating the trade.

As a check we want to make sure each agent has enough sugar and spice to conduct the trade. The way our code is set up all agents with less than 0 sugar or spice should have been removed, but this is a good test to verify our code is working the way we think.

**Part 1**

-----------------------------------------

Now that we have two traders we need to calculate their marginal rate of substitution or MRS, if you want a specific explanation from growing artifical societies you can check on page 102 of the book. However, briefly MRS is the amount of a good that a consumer is willing to consume compared to another good, as long as the new good is equally satisfying (Investopedia, 2022). So we add a helper function for this. The MRS is the amount of spice divided by the metabolism of the amount of spice and the amount of sugar divided by the metabolism of the amount of sugar.  This means that if the agent's MRS is greater than 1 - the numerator is larger then the agent is willing to give up spice, but if the denominator is larger then the agent is willing to trade sugar. 

**Part 2**

---------------------------------------------------

Knowing this we can call the function for each agent in the bilateral trade and run a print statement. 

And it looks like it is working the way we want. Now we are following page 105 in the book, so if the MRS are equal then we end as they will both want to trade the same thing if not we continue, as we are comparing to floating numbers as previous we do not want to do an actual equal but see if the numbers are close. 

**Part 3**

----------------------------------------------------

Now that we have identified the MRS of each agent and identified that they are not close, the next item is to calculate the price. To do this we calculate the geometric mean, which is the square root of the agents two marignal rates of substitution multiplied together. 

We then can print the price and everything appears to be working. 

In the next session we will then conduct the trade. 


In [7]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 
      self.prices = []

  
    def get_sugar(self, pos): #part 2
      '''
      used in:
         get_sugar_amount()
         eat()

      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Sugar:
              return agent
      return None
    
    
    def get_sugar_amount(self, pos):
      '''
      used in move()
      '''
      
      sugar_patch = self.get_sugar(pos)
      if sugar_patch:
          return sugar_patch.amount
      return 0
    
    
    def get_spice(self, pos):
      '''
      used in get_spice_amount
      '''
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for agent in this_cell:
          if type(agent) is Spice:
              return agent
      return None
        
    
    def get_spice_amount(self, pos):
        '''
        used in move()
        '''

        spice_patch = self.get_spice(pos)
        if spice_patch: 
          return spice_patch.amount
        return 0
  
    def get_trader(self, pos):
      '''
      used in trade_with neighbors()
      '''
      this_cell =self.model.grid.get_cell_list_contents(pos)

      for agent in this_cell: 
        if isinstance(agent, Trader):
          return agent
    
    
    def is_occupied(self, pos): 
      '''
      Helper function for move()
      '''
      
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell:
          if isinstance(a, Trader) and a.pos != self.pos:
              return True
      return False
       
    def calculate_welfare(self, sugar, spice): #This will be built to be like the exmaple later with none, but right now it wouldn't make any sense
      '''
      Helper function for move
      '''

      #calculate total resources
      m_total = self.metabolism_sugar + self.metabolism_spice
      #Cobb-Douglas functional form
      return sugar** (self.metabolism_sugar/m_total) * spice ** (
          self.metabolism_spice/m_total)
      
    def is_starved(self): 
      '''
      helpfer function for maybe_die()
      '''

      return (self.sugar <= 0) or (self.spice <=0)

    #Part 1
    def calculate_MRS(self): 
      '''
      used in trade()
      '''

      return (self.spice/self.metabolism_spice) / (self.sugar/self.metabolism_sugar)
    
    
    
    def trade(self, other):
      '''
      used in trade_with_neighbors()
      '''
      #Part 0
      #sanity check to verify code is working the way we think
      assert self.sugar > 0
      assert self.spice > 0
      assert other.sugar > 0 
      assert other.spice > 0 

      #Part 1
      mrs_self = self.calculate_MRS()
      mrs_other = other.calculate_MRS()

      #Part 2
      if math.isclose(mrs_self, mrs_other, rel_tol=1e-2): 
        #print("it was close")
        return 

      #Part 3
      p = math.sqrt(mrs_self*mrs_other)
      print(p)


    ###########################################################################
    #                                                                         #
    #                            MAIN FUNCTIONS                               #
    #                                                                         #
    ###########################################################################

    def move(self): 
           
      # 1. Get neighbors within vision

      neighbors = [i 
                   for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore,True, self.vision
                       )
                   if not self.is_occupied(i)
      ]

      # 2. Find the patch which porduce the maximum welfare. 

      welfares = [
            self.calculate_welfare(
                self.sugar + self.get_sugar_amount(pos),
                self.spice + self.get_spice_amount(pos),
            )
            for pos in neighbors] #part 2

      #find the highest welfare in the cell 
      max_welfare = max(welfares)
      #get the index of maximal welfare cells
      candidate_indices = [i for i in range(len(welfares))
                             if math.isclose(welfares[i], max_welfare,
                                             rel_tol=1e-02)]
      #convert index to positions of those cells
      candidates = [neighbors[i] for i in candidate_indices]
     
      
      # 3. Find the nearest patch among the candidate.
      min_dist = min([get_distance(self.pos, pos) for pos in candidates])
      final_candidates = [
          pos
          for pos in candidates
          if math.isclose(get_distance(self.pos, pos),min_dist, rel_tol=1e-02)
      ]
      self.random.shuffle(final_candidates)

      # 4. Move agent.
      self.model.grid.move_agent(self, final_candidates[0])



    def eat(self):
      #get sugar  #part 1
      #print(self.sugar)
      sugar_patch = self.get_sugar(self.pos)
      #print(sugar_patch)
      if sugar_patch:
        self.sugar = self.sugar - self.metabolism_sugar + sugar_patch.amount
        sugar_patch.amount = 0
        #print(self.sugar, sugar_patch.amount)
      
      #get_spice part 2
      spice_patch = self.get_spice(self.pos)
      #print(spice_patch)
      if spice_patch: 
        self.spice = self.spice - self.metabolism_spice + spice_patch.amount
        spice_patch.amout = 0
        #print(self.spice, spice_patch.amount)

    def maybe_die(self): 

      if self.is_starved(): 
        print(self.unique_id, self.model.schedule.get_type_count(Trader))
        self.model.grid.remove_agent(self) 
        self.model.schedule.remove(self)
        print(self.unique_id, self.model.schedule.get_type_count(Trader))


  
    def trade_with_neighbors(self):
      
      neighbor_agents =[self.get_trader(pos) for pos in self.model.grid.get_neighborhood(
          self.pos, self.moore, False, radius=self.vision) if self.is_occupied(pos)]
        
      if len(neighbor_agents) == 0:
            return []
      
      for a in neighbor_agents: 
        if a: 
          self.trade(a)


In [8]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min = 1,
                 metabolism_max = 5, vision_min = 1, vision_max =5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population 
    self.endowment_min = endowment_min 
    self.endowment_max = endowment_max 
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

    sugar_distribution = np.genfromtxt("/content/drive/MyDrive/sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) 
      y = self.random.randrange(self.height) 
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.metabolism_min, self.metabolism_max)
      metabolism_spice = self.random.uniform(self.metabolism_min, self.metabolism_max)
      #add vision
      vision =int(self.random.uniform(self.vision_min, self.vision_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  
  #part 2
  def randomize_traders(self): 
    """
    Helper function for step()

    updates list of agents used in step function
    """
    
    Traders = self.schedule.agents_by_type[Trader].values()
    Trader_shuffle = list(Traders)
    self.random.shuffle(Trader_shuffle)

    return Trader_shuffle
  
  
  def step(self): 
    
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()
        
    #part t
    Trader_shuffle = self.randomize_traders()
    
    for agent in Trader_shuffle: 
      agent.move() 
      agent.eat() 
      agent.maybe_die() 

    '''
    #demo error part 1
    for agent in Trader_shuffle:
      agent.move()
    '''
    #part 2
    Trader_shuffle = self.randomize_traders()
    

    #part 2
    for agent in Trader_shuffle: 
      #agent.trade_with_neighbors()
      agent.prices = agent.trade_with_neighbors()
    

    self.schedule.time +=1
    self.schedule.steps += 1
    

  
  def run_model(self, step_count=1000):
    
    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

## Model Run 


In [9]:
model = SugarscapeG1mt()
model.run_model(step_count=1) 



1.4190395207099769
1.4177487386216279
0.8565081505566597
0.686353876026319
1.1737833095344106
0.6600156733630179
0.4886131912515896
0.8902284614366114
1.075777328306287
0.8282073317079355
1.0842652499810148
1.6832897909882207
1.0284861816401953
0.42574354296880607
4.568666136955749
1.345316395826983
1.423159569028437
0.6028319249146176
0.7595045835662425
0.707901777514125
1.0619087231140774
0.5371919633989463
0.9632685976691877
4.568666136955749
2.4713176637733567
1.8216447966918237
1.9122567096626335
0.9786487185193257
0.7145323163748575
0.8670762634896332
0.42574354296880607
1.9122567096626335
1.6338393333191568
0.6402987552046595
1.494512792815013
1.1372273262709487
0.76364210406106
1.2726775548584535
0.4365586371172691
0.76364210406106
1.6338393333191568
0.8670762634896332
0.713871524753466
0.5371919633989463
0.6124365597412063
0.7359139688893683
0.5117095605836233
0.5998061092321272
0.6693410951472375
1.9249061719435847
0.7359139688893683
0.6387672028694837
0.5744753798185841
1.07